### train_data ###

In [13]:
import json

In [14]:
from collections import Counter

In [15]:
from utils import *

### 确定MAX_LENGTH ###

In [16]:

with open('../bishe_e1/data/mytrain.json','r',encoding='gbk') as f2:
    lines = f2.readlines()
#     raw_tra = json.load(f2)
f2.close()

length = len(lines)

label_list = []
count_list = []
for i in range(length):
    sentence = json.loads(lines[i])
    label = sentence['relationMentions'][0]['label']
    label_list.append(label)
    sent_con = sentence['sentText'].strip()
    count_list.append(len(sent_con))
    

# sentence = json.loads(lines[10])
print('案件总数：',length)
assert len(label_list) == length

from collections import Counter

print(Counter(label_list))

print()
print(Counter(count_list))

#### 可以确定每一个sentence max_length = 300

案件总数： 2201
Counter({'provide_shelter_for': 835, 'sell_drugs_to': 472, 'posess': 470, 'traffic_in': 424})

Counter({90: 29, 104: 25, 152: 24, 124: 22, 82: 22, 256: 20, 127: 20, 114: 19, 184: 19, 144: 19, 111: 18, 115: 18, 128: 17, 136: 17, 148: 17, 150: 17, 117: 17, 122: 17, 176: 17, 154: 16, 118: 16, 84: 16, 86: 16, 106: 16, 206: 16, 166: 16, 88: 16, 146: 16, 95: 16, 74: 15, 195: 15, 205: 15, 140: 15, 138: 15, 79: 15, 175: 15, 147: 15, 235: 15, 121: 15, 254: 14, 107: 14, 125: 14, 66: 14, 243: 14, 123: 14, 223: 14, 165: 14, 162: 13, 181: 13, 139: 13, 194: 13, 142: 13, 134: 13, 110: 13, 93: 13, 99: 13, 89: 13, 87: 12, 151: 12, 156: 12, 168: 12, 153: 12, 64: 12, 340: 12, 77: 12, 132: 12, 61: 12, 137: 11, 78: 11, 192: 11, 65: 11, 143: 11, 167: 11, 185: 11, 221: 11, 116: 11, 67: 11, 204: 11, 179: 10, 191: 10, 119: 10, 112: 10, 161: 10, 220: 10, 244: 10, 135: 10, 233: 10, 237: 10, 155: 10, 96: 10, 75: 10, 85: 10, 270: 10, 169: 10, 73: 10, 101: 9, 164: 9, 187: 9, 199: 9, 186: 9, 97: 9, 193: 9

### json --> pk

In [17]:
MAX_SENT_LENGTH = 300#最大句长


In [18]:
def make_tag_set(tag_set, relation_label):##生成标签集合tag_set
    if relation_label == "None":
        return
    for pos in "BIES":
        for role in "12":
            tag_set.add("-".join([pos, relation_label, role]))#pos-relation_label-role


In [19]:
# len(charset)    ##2194
# charset[-1]   ### '<unk>'
#charset[-2]  ###'<pad>'
#charset[2192] ### '<pad>'
#charset[2193] ### '<unk>'

In [20]:
relation_labels = Index()
entity_labels = Index()
tag_set = Index()

tag_set.add("O")

0

In [21]:
# max_tmp = 0
# min_tmp = 100
# max_span = 0

# for line in lines:
#     line = line.strip()
#     line = json.loads(line)
    
#     e1 = int(line["relationMentions"][0]['e1start'])
#     e2 = int(line["relationMentions"][0]['e21start'])
    
#     if e2 > 300 :
#         print(e1)
#         print(e2)
#         print()
#     if len(line["sentText"].strip().strip('"')) > 300:
#         e1 = int(line["relationMentions"][0]['e1start'])
#         e2 = int(line["relationMentions"][0]['e21start'])
#         if e2-e1 > max_span:
#             max_span = e2-e1
        
# #         if e2-e1 == 242:
# #             print(line)
        
#         if e2 > max_tmp:
#             max_tmp = e2
            
#         if e1 <min_tmp:
#             min_tmp = e1
            
        
#         if e2 > 350:
#             print('Hello')
#         if e2 > 256:
#             print('World')
#         print(line["sentText"].strip().strip('"'))
#         print(line["relationMentions"][0]['e1start'])
#         print(line["relationMentions"][0]['e21start'])
# print(max_tmp,min_tmp,max_span)

In [22]:
def Supdate_tag_seq(em_text, e_start, sentence_text, relation_label, role, tag_set, tags_idx):#给句子中的实体打标签 得到tags_idx
    overlap = False
    wrongstart=0
    
    start=e_start
    if start==-1:
        wrongstart+=1
    tag = "-".join(["S", relation_label, str(role)])
    if len(em_text) == 1:
        #print(em_text,e_start)
        if tags_idx[start] != tag_set["O"]:
            overlap = True
        tags_idx[start] = tag_set[tag]#S-...-...
    else:
        tag = "B" + tag[1:]
        if tags_idx[start] != tag_set["O"]:
            overlap = True
        tags_idx[start] = tag_set[tag]
        tag = "E" + tag[1:]
        end = start + len(em_text) - 1 #计算实体词的长度
        if tags_idx[end] != tag_set["O"]:
            overlap = True
        tags_idx[end] = tag_set[tag]
        tag = "I" + tag[1:]
        for index in range(start + 1, end):
            if tags_idx[index] != tag_set["O"]:
                overlap = True
            tags_idx[index] = tag_set[tag]
            
    #print("wrong start:",wrongstart)
    return overlap              #返回重复的已标记过的实体



In [25]:
#prepare_data_set(fin, charset, vocab, relation_labels, entity_labels, tag_set, train, fout)
#参数：输入的.json文件，字符集对象，单词集合对象（已导入词典），关系标签，实体标签，实体-关系标签集合，train/test list，overlap.txt文件
def prepare_data_set(fin, charset, relation_labels, entity_labels, tag_set, dataset, fout):##预处理数据集  json->.pk

    num_overlap = 0
    for line in fin:
        overlap = False
        line = line.strip()
        if not line:
            continue  #结束则跳出循环
        sentence = json.loads(line)#将json数据转换成python对象  这里是转换为了字典
        for relation_mention in sentence["relationMentions"]:
            relation_labels.add(relation_mention["label"])  #记录关系类型
            make_tag_set(tag_set, relation_mention["label"]) #生成标签集合 BIES-relation-12
        for entity_mention in sentence["entityMentions"]:
            entity_labels.add(entity_mention["label"])  #记录实体类型
        
        sentence_text=list((sentence["sentText"].strip().strip('"')))
        
        
#         if length_sent > MAX_SENT_LENGTH:#设置了最大句长
# #             continue
#             sentence_text = sentence_text[50:350]
        length_sent = len(sentence_text) #该条文本长度
        
        sentence_idx = prepare_sequence(sentence_text, charset)  #charset是词和id的映射 将sentence中的每个词转为对应的id值得到id的list
        tags_idx = [tag_set["O"]] * length_sent #length_sent是该句子的长度  初始化为全0
        
        for relation_mention in sentence["relationMentions"]:
            if relation_mention["label"] == "None":
                continue
#             relation_label = relation_mention['label']
            em1_text = list(relation_mention["em1Text"])  #英文中 对phrase分词
            e1_s=int(relation_mention["e1start"])
            if e1_s==-1:
                wrongstart+=1
#             tag = "-".join(["S", relation_label, str(role)])           
            res1 = Supdate_tag_seq(em1_text, e1_s, sentence_text, relation_mention["label"], 1, tag_set, tags_idx)#对句子中的实体打标签
            em2_text = list(relation_mention["em2Text"])
            e2_s=int(relation_mention["e21start"])
            res2 = Supdate_tag_seq(em2_text, e2_s, sentence_text, relation_mention["label"], 2, tag_set, tags_idx)
            
            if res1 or res2:
                num_overlap += 1 #重复
                overlap = True
#         print(tags_idx)
        
        
        dataset.append((sentence_idx, tags_idx)) #dateset是train/test  包含三部分 句子中每个词的id 句子中每个词对应的char的id 句子对应的实体id        
        fout.write(''.join(sentence_text)+'\n')
#         if overlap:
#             fout.write(line+"\n")
            
#         break
    return num_overlap  #返回重复的已标记过的语句

In [26]:

Chinesecorpus=open('../bishe_e1/data/mycorpus.txt','rt',encoding='utf-8').read()
cs_list = list(Chinesecorpus)
cs_set_list = list(set(cs_list))
cs_set_list.sort(key = cs_list.index)
corpus_char=''.join(cs_set_list)    ###2192
charset=ChineseCharset(corpus_char)

relation_labels = Index()
entity_labels = Index()
tag_set = Index()
tag_set.add("O")

with open("../data/my_train.txt", "wt", encoding="utf-8") as fout:
    train = []
#     train_seg=[]
#     with open(r'../data/mytrain_seg.txt','wt',encoding='utf-8') as fseg:
    with open('../bishe_e1/data/mytrain.json', 'rt', encoding='utf-8') as fin:##对训练集进行预处理

        res = prepare_data_set(fin, charset,relation_labels, entity_labels, tag_set, train, fout)

    save(train, '../data/my_train.pk')
        
#         fseg.write('\n'.join(train_seg))
    print('begin...')
with open("../data/my_test.txt", "wt", encoding="utf-8") as fout:
    test=[]

    with open(r'..\bishe_e1\data\mytest.json', 'rt', encoding='utf-8') as fin:##对训练集进行预处理
#             res = prepare_data_set(fin, charset, vocab, relation_labels, entity_labels, tag_set, test, fout,test_seg)##参数：输入的.json文件，字符集对象，单词集合对象，关系标签，实体标签，实体-关系标签集合，train/test list，overlap.txt文件
        res = prepare_data_set(fin, charset,relation_labels, entity_labels, tag_set, test, fout)
    save(test, '../data/my_test.pk')
#         fseg.write('\n'.join(test_seg))

    print('over...')
relation_labels.save('../data/my_relation_labels.txt')
entity_labels.save('../data/my_entity_labels.txt')
tag_set.save("../data/my_tag2id.txt")

begin...
over...


### 测试正确性 ###

In [38]:
Chinesecorpus=open('../bishe_e1/data/mycorpus.txt','rt',encoding='utf-8').read()
cs_list = list(Chinesecorpus)
cs_set_list = list(set(cs_list))
cs_set_list.sort(key = cs_list.index)
corpus_char=''.join(cs_set_list)    ###2192
charset=ChineseCharset(corpus_char)
charset[1186]

'核'

In [34]:
import pickle

with open('../data/my_train.pk', 'rb') as f1:
    train_data = pickle.load(f1)
f1.close()

print(len(train_data)) ## 2201
print(len(train_data[0][0])) ### 74
print(len(train_data[0][1])) ### 74


2201
74
74


In [37]:
charset[1]

'诉'

In [35]:
train_data[0][0]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 9,
 9,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 6,
 19,
 20,
 21,
 22,
 23,
 23,
 24,
 25,
 26,
 23,
 27,
 28,
 23,
 27,
 29,
 23,
 23,
 27,
 30,
 23,
 27,
 31,
 23,
 23,
 32,
 22,
 23,
 23,
 33,
 15,
 34,
 35,
 36,
 6,
 37,
 38,
 39,
 14,
 40,
 41,
 39,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52]

In [129]:
# with open('../data/my_test.pk', 'rb') as f1:
#     train_data = pickle.load(f1)
# f1.close()

# print(len(train_data)) ## 550
# print(len(train_data[0][0])) ### 170
# print(len(train_data[0][1])) ### 170

550
170
170


<hr />

### 简介 ###
上面为正式的预处理<br>
下面是做出上述预处理的一些测试程序

In [14]:
with open('../bishe_e1/data/mytrain.json', 'rt', encoding='utf-8') as fin:##对训练集进行预处理
    lines = fin.readlines()
fin.close()

In [15]:
line = lines[0].strip()
sentence = json.loads(line)

#### 涉及多个关系时，这样做比较合理
for relation_mention in sentence["relationMentions"]:
    relation_labels.add(relation_mention["label"])  #记录关系类型
    make_tag_set(tag_set, relation_mention["label"]) #生成标签集合 BIES-relation-12
for entity_mention in sentence["entityMentions"]:
    entity_labels.add(entity_mention["label"])  #记录实体类型
#sentence_text = sentence["sentText"].strip().strip('"').split()#保存文本为单词list
sentence_text=list((sentence["sentText"].strip().strip('"')))

length_sent = len(sentence_text) #该条文本长度

if length_sent > MAX_SENT_LENGTH:#设置了最大句长
    print("Too Long !!!")
#     continue

sentence_idx = prepare_sequence(sentence_text, charset)

tags_idx = [tag_set["O"]] * length_sent #length_sent是该句子的长度  初始化为全0

relation_mention

In [20]:
def prepare_sequence(seq, to_idx):
    return [to_idx[key] for key in seq]

In [79]:
print(tags_idx)
for i in range(len(tags_idx)):
    tag = tags_idx[i]
    if tag != 0:
        print(i)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
25
26
27
47
48
49


In [49]:
wrongstart = 0
relation_label = relation_mention["label"]
role = 1
for relation_mention in sentence["relationMentions"]:
    if relation_mention["label"] == "None":
        continue
    em1_text = list(relation_mention["em1Text"])  #英文中 对phrase分词
    e1_s=int(relation_mention["e1start"])
    if e1_s==-1:
        wrongstart+=1
    tag = "-".join(["S", relation_label, str(role)])
    print(tag)
#     print(tag_set[14])
    
    assert len(tags_idx) == len(sentence_text)
    res1 = Supdate_tag_seq(em1_text, e1_s, sentence_text, relation_mention["label"], 1, tag_set, tags_idx)#对句子中的实体打标签
    
    em2_text = list(relation_mention["em2Text"])
    e2_s=int(relation_mention["e21start"])
    res2 = Supdate_tag_seq(em2_text, e2_s, sentence_text, relation_mention["label"], 2, tag_set, tags_idx)
#     if res1 or res2:
#         num_overlap += 1 #重复
#         overlap = True

S-provide_shelter_for-1


In [50]:
tag_set[1],tag_set[2]

tag_set[3],tag_set[4]

tag_set[5],tag_set[6]

('B-provide_shelter_for-1', 'B-provide_shelter_for-2')

In [59]:
print(tags_idx)
for i in range(len(tags_idx)):
    tag = tags_idx[i]
    if tag != 0:
        print(i)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
25
26
27
47
48
49


In [57]:
print(sentence_text)

['公', '诉', '机', '关', '指', '控', '，', '2', '0', '1', '5', '年', '1', '1', '月', '份', '的', '一', '天', '晚', '上', '，', '被', '告', '人', '简', '某', '某', '容', '留', '陈', '某', '、', '吴', '某', '、', '杨', '某', '某', '、', '张', '某', '、', '伍', '某', '某', '在', '简', '某', '某', '家', '一', '楼', '卧', '室', '，', '以', '烫', '吸', '的', '方', '式', '吸', '食', '甲', '基', '苯', '丙', '胺', '（', '冰', '毒', '）', '。']
